In [3]:
import pandas as pd
import numpy as np
from fastembed import TextEmbedding
import langdetect
from textblob import TextBlob
from sklearn.preprocessing import LabelEncoder
from sklearn.cluster import KMeans
import ast

In [4]:
import pandas as pd
df = pd.read_csv('songs_full.csv')
df.head()

,id,title,artist,album,thumbnail,duration,related_videos,views,is_private,is_unlisted,category,publish_date,upload_date,length_seconds,is_family_safe,available_countries
0,qxsQe2EAk40,City Slums (feat. DIVINE),Raja Kumari,NaN,https://i.ytimg.com/vi/qxsQe2EAk40/sddefault.j...,3:45,[],86844848,False,NaN,Music,2017-10-03T23:29:22-07:00,2017-10-03T23:29:22-07:00,224.0,NaN,249
1,FEvrFe7GWwQ,Roots (feat. Raja Kumari),DIVINE,NaN,https://i.ytimg.com/vi/FEvrFe7GWwQ/sddefault.j...,2:44,[],9267272,False,NaN,Music,2018-07-26T23:29:39-07:00,2018-07-26T23:29:39-07:00,164.0,NaN,249
2,ZYu7LcHi84M,Mere Bhole Baba,Pardhaan,NaN,https://i.ytimg.com/vi/ZYu7LcHi84M/sddefault.j...,3:55,[],13561347,False,NaN,Music,2018-02-10T21:30:01-08:00,2018-02-10T21:30:01-08:00,234.0,NaN,249
3,KgfKTfJ70zI,Don't Hold Back 2.0 by JACK & JONES FT. Ranvee...,JACK & JONES India,NaN,https://i.ytimg.com/vi/KgfKTfJ70zI/sddefault.j...,3:00,[],13116755,False,NaN,Music,2017-05-18T17:15:34-07:00,2017-05-18T17:15:34-07:00,180.0,NaN,249
4,NU3xm6N7tzw,Kaam 25,DIVINE,NaN,https://i.ytimg.com/vi/NU3xm6N7tzw/sddefault.j...,2:42,[],96036911,False,NaN,Music,2018-06-14T22:30:02-07:00,2018-06-14T22:30:02-07:00,161.0,NaN,249


In [5]:
df.columns

Index(['id', 'title', 'artist', 'album', 'thumbnail', 'duration',
       'related_videos', 'views', 'is_private', 'is_unlisted', 'category',
       'publish_date', 'upload_date', 'length_seconds', 'is_family_safe',
       'available_countries'],
      dtype='object')

In [6]:
# Fill missing values
df['views'] = df['views'].fillna(0)
df['length_seconds'] = df['length_seconds'].fillna(df['duration'])
df['category'] = df['category'].fillna('Unknown')
df['album'] = df['album'].fillna('Unknown')
df['publish_date'] = df['publish_date'].fillna(df['upload_date'])
df['title'] = df['title'].fillna('')
df['artist'] = df['artist'].fillna('')

In [7]:
# Convert dates to year
def extract_year(x):
    try:
        return int(str(x)[:4])
    except:
        return None

df['release_year'] = df['publish_date'].apply(extract_year)
df['release_year'] = df['release_year'].fillna(df['release_year'].median())

In [8]:
df['popularity'] = np.log1p(df['views'])  # log scale

In [9]:
def detect_lang(text):
    try:
        return langdetect.detect(text)
    except:
        return "unknown"

df['language'] = df['title'].apply(detect_lang)

# Encode language
le_lang = LabelEncoder()
df['language_encoded'] = le_lang.fit_transform(df['language'])

In [10]:
def get_sentiment(text):
    analysis = TextBlob(text)
    return analysis.polarity, analysis.subjectivity

df['sentiment_polarity'], df['sentiment_subjectivity'] = zip(
    *df['title'].apply(get_sentiment)
)

In [11]:
le_cat = LabelEncoder()
df['category_encoded'] = le_cat.fit_transform(df['category'])

In [13]:
from fastembed import TextEmbedding
import numpy as np

# Load model
model = TextEmbedding()

# Create text blob
df['text_blob'] = df['title'] + " " + df['artist'] + " " + df['album']

# FastEmbed returns a generator → convert to list THEN np array
emb_list = list(model.embed(df['text_blob'].tolist()))
embeddings = np.array(emb_list)

# Now embeddings.shape works
embedding_dim = embeddings.shape[1]

# Add embedding columns
for i in range(embedding_dim):
    df[f"emb_{i}"] = embeddings[:, i]

/var/folders/r_/cwd775rd0td5_q92qnzfdwt80000gn/T/ipykernel_44039/2019190819.py:19: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f"emb_{i}"] = embeddings[:, i]
/var/folders/r_/cwd775rd0td5_q92qnzfdwt80000gn/T/ipykernel_44039/2019190819.py:19: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f"emb_{i}"] = embeddings[:, i]
/var/folders/r_/cwd775rd0td5_q92qnzfdwt80000gn/T/ipykernel_44039/2019190819.py:19: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which h

In [14]:
kmeans = KMeans(n_clusters=15, random_state=42)
df['genre_cluster'] = kmeans.fit_predict(embeddings)

/opt/anaconda3/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
/var/folders/r_/cwd775rd0td5_q92qnzfdwt80000gn/T/ipykernel_44039/2831332273.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['genre_cluster'] = kmeans.fit_predic

In [15]:
df['available_countries'] = df['available_countries'].fillna(0)

In [16]:
def fix_list(x):
    try:
        if isinstance(x, list): return x
        return ast.literal_eval(x)
    except:
        return []

df['related_videos'] = df['related_videos'].apply(fix_list)

In [21]:
final_columns = [
    "id", "title", "artist", "album", "thumbnail",
    "popularity", "length_seconds", "release_year",
    "language_encoded", "sentiment_polarity",
    "sentiment_subjectivity", "category_encoded",
    "genre_cluster", "available_countries",
    "related_videos"
] + [f"pca_emb_{i}" for i in range(64)] #pca_emb_54

df_final = df[final_columns]

In [19]:
from sklearn.decomposition import PCA

# embeddings = np.array(...) from your fastembed step

n_components = 64  # try 32 or 64
pca = PCA(n_components=n_components, random_state=42)

embeddings_reduced = pca.fit_transform(embeddings)

print("Original embedding shape:", embeddings.shape)
print("PCA reduced shape:", embeddings_reduced.shape)

# Add PCA components to dataframe
for i in range(n_components):
    df[f"pca_emb_{i}"] = embeddings_reduced[:, i]

Original embedding shape: (256, 384)
PCA reduced shape: (256, 64)


/var/folders/r_/cwd775rd0td5_q92qnzfdwt80000gn/T/ipykernel_44039/2180495280.py:15: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f"pca_emb_{i}"] = embeddings_reduced[:, i]
/var/folders/r_/cwd775rd0td5_q92qnzfdwt80000gn/T/ipykernel_44039/2180495280.py:15: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f"pca_emb_{i}"] = embeddings_reduced[:, i]
/var/folders/r_/cwd775rd0td5_q92qnzfdwt80000gn/T/ipykernel_44039/2180495280.py:15: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.ins

In [22]:
df_final

,id,title,artist,album,thumbnail,popularity,length_seconds,release_year,language_encoded,sentiment_polarity,...,pca_emb_54,pca_emb_55,pca_emb_56,pca_emb_57,pca_emb_58,pca_emb_59,pca_emb_60,pca_emb_61,pca_emb_62,pca_emb_63
0,qxsQe2EAk40,City Slums (feat. DIVINE),Raja Kumari,Unknown,https://i.ytimg.com/vi/qxsQe2EAk40/sddefault.j...,18.279634,224.0,2017.0,6,0.00,...,0.001011,-0.091985,0.025934,0.012378,0.108026,0.024659,-0.010928,-0.031891,-0.004023,0.046576
1,FEvrFe7GWwQ,Roots (feat. Raja Kumari),DIVINE,Unknown,https://i.ytimg.com/vi/FEvrFe7GWwQ/sddefault.j...,16.042000,164.0,2018.0,8,0.00,...,-0.050557,-0.059383,0.070665,-0.006106,0.050605,-0.024939,-0.040476,-0.025911,0.042549,-0.005365
2,ZYu7LcHi84M,Mere Bhole Baba,Pardhaan,Unknown,https://i.ytimg.com/vi/ZYu7LcHi84M/sddefault.j...,16.422734,234.0,2018.0,6,-0.50,...,-0.046698,0.096364,-0.009431,0.009336,-0.015394,-0.011767,-0.010683,0.032495,0.028867,0.022979
3,KgfKTfJ70zI,Don't Hold Back 2.0 by JACK & JONES FT. Ranvee...,JACK & JONES India,Unknown,https://i.ytimg.com/vi/KgfKTfJ70zI/sddefault.j...,16.389401,180.0,2017.0,6,0.00,...,0.012730,-0.061857,-0.084588,-0.059387,-0.128752,-0.088900,-0.066034,0.011328,-0.038942,-0.028746
4,NU3xm6N7tzw,Kaam 25,DIVINE,Unknown,https://i.ytimg.com/vi/NU3xm6N7tzw/sddefault.j...,18.380243,161.0,2018.0,14,0.00,...,0.004801,0.041126,-0.037704,-0.034931,0.108884,-0.015100,0.028189,0.045116,-0.077571,-0.023848
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
251,zcZ6J5tb9oQ,Digital Silence,Peter McPoland,Digital Silence,https://lh3.googleusercontent.com/ZdhzDP8hhubj...,16.784045,198.0,2023.0,15,0.00,...,0.044509,0.108244,0.058749,-0.106034,0.079370,0.048185,-0.017657,0.063241,0.048011,0.023232
252,FHypEgzJFlc,Angel Of Small Death & The Codeine Scene,Hozier,Hozier (Special Edition),https://lh3.googleusercontent.com/DzA6tmNob8pt...,14.782684,219.0,2018.0,5,-0.25,...,0.040257,0.000172,-0.046878,0.066882,-0.078848,0.049561,-0.008741,0.104634,0.008386,-0.112623
253,LU1Yxni4K5E,Breakfast,Dove Cameron,Breakfast,https://lh3.googleusercontent.com/_Un9jF5pa96o...,16.429565,148.0,2022.0,6,0.00,...,-0.099627,-0.065259,-0.062519,-0.038779,0.011692,-0.033260,0.036782,-0.046339,-0.015902,-0.095997
254,l_2Mqypgx0w,Devil Is A Woman,Cloudy June,Devil Is A Woman,https://lh3.googleusercontent.com/-yltlUNzyqBQ...,14.241287,192.0,2022.0,6,0.00,...,-0.000276,-0.072236,-0.084432,-0.033302,0.001572,-0.004110,0.035253,0.102925,0.025629,-0.005216


In [23]:
df_final.columns

Index(['id', 'title', 'artist', 'album', 'thumbnail', 'popularity',
       'length_seconds', 'release_year', 'language_encoded',
       'sentiment_polarity', 'sentiment_subjectivity', 'category_encoded',
       'genre_cluster', 'available_countries', 'related_videos', 'pca_emb_0',
       'pca_emb_1', 'pca_emb_2', 'pca_emb_3', 'pca_emb_4', 'pca_emb_5',
       'pca_emb_6', 'pca_emb_7', 'pca_emb_8', 'pca_emb_9', 'pca_emb_10',
       'pca_emb_11', 'pca_emb_12', 'pca_emb_13', 'pca_emb_14', 'pca_emb_15',
       'pca_emb_16', 'pca_emb_17', 'pca_emb_18', 'pca_emb_19', 'pca_emb_20',
       'pca_emb_21', 'pca_emb_22', 'pca_emb_23', 'pca_emb_24', 'pca_emb_25',
       'pca_emb_26', 'pca_emb_27', 'pca_emb_28', 'pca_emb_29', 'pca_emb_30',
       'pca_emb_31', 'pca_emb_32', 'pca_emb_33', 'pca_emb_34', 'pca_emb_35',
       'pca_emb_36', 'pca_emb_37', 'pca_emb_38', 'pca_emb_39', 'pca_emb_40',
       'pca_emb_41', 'pca_emb_42', 'pca_emb_43', 'pca_emb_44', 'pca_emb_45',
       'pca_emb_46', 'pca_emb_47',

In [34]:
df_final.to_csv("final_songs.csv", index=False)

print("Saved cleaned dataset with numeric features → final_songs.csv")
print("Clean dataset shape:", df_final.shape)

Saved cleaned dataset with numeric features → final_songs.csv
Clean dataset shape: (256, 79)


In [32]:
df_final[['popularity',
    'length_seconds',
    'release_year',
    'language_encoded',
    'sentiment_polarity',
    'sentiment_subjectivity',
    'category_encoded',
    'genre_cluster',
    'available_countries']].dtypes

popularity                float64
length_seconds             object
release_year              float64
language_encoded            int64
sentiment_polarity        float64
sentiment_subjectivity    float64
category_encoded            int64
genre_cluster               int32
available_countries         int64
dtype: object

In [33]:
def duration_to_seconds(x):
    """
    Convert '3:55' → 235 seconds.
    Convert None or invalid → 0.
    """
    try:
        if isinstance(x, (int, float)):
            return x
        if isinstance(x, str) and ":" in x:
            mins, secs = x.split(":")
            return int(mins) * 60 + int(secs)
        return int(x)
    except:
        return 0
df_final["length_seconds"] = df_final["length_seconds"].apply(duration_to_seconds)

/var/folders/r_/cwd775rd0td5_q92qnzfdwt80000gn/T/ipykernel_44039/2535251973.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final["length_seconds"] = df_final["length_seconds"].apply(duration_to_seconds)


In [36]:
import pickle
import networkx as nx
import plotly.graph_objects as go

# Load graph
with open("song_graph.pkl", "rb") as f:
    G = pickle.load(f)

# Pick a node to visualize
node = list(G.nodes())[10]

# -----------------------------
# LIMIT TO TOP-K strongest neighbors
# -----------------------------
TOP_K = 8

neighbors = sorted(
    G[node].items(),
    key=lambda x: x[1]["weight"],
    reverse=True
)[:TOP_K]

neighbor_nodes = [node] + [n for n, d in neighbors]
sub = G.subgraph(neighbor_nodes)

# -----------------------------
# Compute positions
# -----------------------------
pos = nx.spring_layout(sub, seed=42, k=0.4)

# Extract coordinates
x_nodes = [pos[n][0] for n in sub.nodes()]
y_nodes = [pos[n][1] for n in sub.nodes()]

# Edge coordinates
edge_x = []
edge_y = []
for u, v in sub.edges():
    edge_x += [pos[u][0], pos[v][0], None]
    edge_y += [pos[u][1], pos[v][1], None]

# -----------------------------
# Plotly figure
# -----------------------------
fig = go.Figure()

# Add edges
fig.add_trace(go.Scatter(
    x=edge_x, y=edge_y,
    mode='lines',
    line=dict(width=1, color='gray'),
    hoverinfo='none'
))

# Add nodes
fig.add_trace(go.Scatter(
    x=x_nodes, y=y_nodes,
    mode='markers+text',
    marker=dict(size=40, color='skyblue', line=dict(width=2, color="black")),
    text=[G.nodes[n]["title"][:18] for n in sub.nodes()],
    textposition="bottom center",
    hovertext=[
        f"<b>{G.nodes[n]['title']}</b><br>{G.nodes[n]['artist']}"
        for n in sub.nodes()
    ],
    hoverinfo="text"
))

fig.update_layout(
    title=f"Song Neighborhood (Top {TOP_K} Similar Tracks)",
    showlegend=False,
    width=900,
    height=900,
    xaxis=dict(showgrid=False, zeroline=False, visible=False),
    yaxis=dict(showgrid=False, zeroline=False, visible=False),
)

fig.show()

In [43]:
import pickle

with open("song_graph_1.pkl", "rb") as f:
    G = pickle.load(f)

bad_nodes = [n for n in G.nodes() if not isinstance(n, str)]
bad_nodes

[]

In [40]:
import pandas as pd

df = pd.read_csv("final_songs.csv")
df[df['id'].isna()]

,id,title,artist,album,thumbnail,popularity,length_seconds,release_year,language_encoded,sentiment_polarity,...,pca_emb_54,pca_emb_55,pca_emb_56,pca_emb_57,pca_emb_58,pca_emb_59,pca_emb_60,pca_emb_61,pca_emb_62,pca_emb_63
206,NaN,Neele Neele Ambar Par (Male Version) Kalaakaar...,Kishore Kumar,Unknown,https://i.ytimg.com/vi/nEnLt3pasxE/sddefault.j...,0.0,235.0,2021.0,8,0.0,...,-0.030589,-0.109923,0.003226,0.006741,0.070726,0.012245,0.02716,-0.030301,-0.01889,0.068292


In [41]:
df[df['id'].astype(str).str.strip() == ""]

,id,title,artist,album,thumbnail,popularity,length_seconds,release_year,language_encoded,sentiment_polarity,...,pca_emb_54,pca_emb_55,pca_emb_56,pca_emb_57,pca_emb_58,pca_emb_59,pca_emb_60,pca_emb_61,pca_emb_62,pca_emb_63


In [42]:
df = df[df['id'].apply(lambda x: isinstance(x, str) and x.strip() != "" and x.lower() != "nan")]
df.to_csv("final_songs_clean.csv", index=False)

In [45]:
df_final['language_encoded'].nunique()

30